# Preparación de datos

* Eliminar las preguntas que no se usarán
* Agrupa las filas por encuesta (estaban por persona)
* Actualizar valores con IPC

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time

from scipy import stats
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Preguntas:
df_preguntas = pd.read_excel('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Docs/metadata_directory/Preguntas.xlsx')
df_preguntas = df_preguntas[['Npregunta.1', 'Pregunta', 'Filtro', 'Tipo de filtro']]

In [4]:
# Diccionario de datos:
df_dicc = pd.read_csv('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Docs/metadata_directory/DiccionarioPreguntas.csv', sep=";")

In [5]:
# ds de trabajo
df_or = pd.read_csv('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Data/Processed/T0_ECV_161718.csv', sep=";")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (225,251) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_or.head()

,Unnamed: 0,encuesta,barrio,comuna,estrato,hogar,persona,factor vivienda,factor persona,año,p_146,p_147,p_148,p_149,p_150,p_151,p_152,p_153,p_154,p_155,p_156,p_157,p_158,p_159,p_160,p_161,p_162,p_163,p_164,p_165,p_166,p_167,p_168,p_169,p_170,p_171,p_172,p_173,p_174,p_175,...,p_106,p_107,p_108,p_109,p_110,p_111,p_112,p_113,p_114,p_115,p_116,p_117,p_118,p_119,p_120,p_121,p_122,p_124,p_123,p_125,p_126,p_127,p_128,p_129,p_130,p_131,p_132,p_133,p_134,p_135,p_136,p_137,p_138,p_139,p_140,p_141,p_142,p_143,p_144,p_145
0,0,12016.0,LAURELES,LAURELES-ESTADIO,5,1,1,87.544025,96.382979,2016,5,10,4,1,1,1,0,1,1,5,0,9,1,1,2,2,1,1,2,1,1,1,1,2,2,1,1,2,1,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,2,2,2,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88
1,1,12016.0,LAURELES,LAURELES-ESTADIO,5,1,2,87.544025,77.870130,2016,5,10,4,1,1,1,0,1,1,5,0,9,1,1,2,2,1,1,2,1,1,1,1,2,2,1,1,2,1,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,2,-88,-88,2,-88,-88,-99,-99,-99,-99,-99,2,2,2,2,-88,1,2,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88
2,2,12016.0,LAURELES,LAURELES-ESTADIO,5,1,3,87.544025,74.466667,2016,5,10,4,1,1,1,0,1,1,5,0,9,1,1,2,2,1,1,2,1,1,1,1,2,2,1,1,2,1,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,2,-88,-88,2,-88,-88,-99,-99,-99,-99,-99,2,2,2,2,-88,1,2,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88
3,3,12016.0,LAURELES,LAURELES-ESTADIO,5,1,4,87.544025,31.235294,2016,5,10,4,1,1,1,0,1,1,5,0,9,1,1,2,2,1,1,2,1,1,1,1,2,2,1,1,2,1,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,2,-88,-88,2,-88,-88,-99,-99,-99,-99,-99,2,2,2,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88
4,4,12016.0,LAURELES,LAURELES-ESTADIO,5,1,5,87.544025,44.333333,2016,5,10,4,1,1,1,0,1,1,5,0,9,1,1,2,2,1,1,2,1,1,1,1,2,2,1,1,2,1,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88,1,1,2,-88,-88,-99,-99,-99,-99,-99,-99,2,2,2,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88,2,-88


In [7]:
df_or = df_or.drop(['Unnamed: 0'], axis=1)

In [8]:
# funcion para consultar posibles valores de una pregunta:
def consultar_dicc(pregunta):
  print (df_dicc[df_dicc['pregunta'] == pregunta])


# Eliminando y agrupando preguntas

Tras un análisis exploratorio de datos, se encontró que no son necesarias todas las preguntas para el desarrollo del proyecto, se seleccionaron unas (ver archivo [Preguntas.xlsx](https://drive.google.com/file/d/1FS5wjsMnsS_Ygpmcm64Vz1q6G_iSeLSF/view?usp=sharing)).

Las respuestas están por persona, en cada hogar hay más de una persona, por lo tantoes neceesario agruparlas por hogar (la variable encuesta es la que las agrupa). Además, para algunas preguntas se tomó el máximo valor entre las respuestas del hogar, estas se encuentran agrupadas en df_grouped_max. Para otras preguntas se sumó la frecuencia de aparición de la categoria en el hogar, estas se encuentran agrupadas en df_agrup. 


In [9]:
df_grouped_max = df_or.groupby('encuesta')[['p_5', 'p_6', 'p_7', 'p_10', 'p_12', 'p_44', 'p_88',
                                     'p_89', 'p_90', 'p_91', 'p_92', 'p_93', 'p_112', 'p_113',
                                     'p_114', 'p_115', 'p_125', 'p_126', 'p_146','p_147','p_148', 
                                     'p_157','p_158', 'p_162','p_165', 'p_171','p_174','p_178', 
                                     'p_184', 'p_211','p_212', 'p_213', 'p_214', 'p_227', 'p_232', 
                                     'p_233', 'p_234', 'p_235', 'p_236', 'p_237', 'p_238', 'p_239',
                                     'p_240', 'p_241','p_242', 'p_243', 'p_244', 'p_245', 'p_246',
                                     'p_247', 'p_248','p_249', 'p_250', 'p_251', 'p_252', 'p_253', 
                                     'p_254','p_258', 'p_259', 'p_260', 'p_281', 'p_282', 
                                     'p_283', 'p_284', 'p_285','p_286', 'p_312', 'p_313', 'p_314', 
                                     'p_315', 'p_316', 'p_317','p_318', 'p_319', 'p_320', 'p_321',  
                                     'p_322', 'p_323']].max().reset_index()

Para no perder información, a algunas preguntas se les realizó la suma de frecuencias por categoría. Ej: P_15 - Género: 1- Hombre / 2- Mujer

In [10]:
consultar_dicc('p_15')

    pregunta  codigo etiqueta
339     p_15       1   Hombre
340     p_15       2    Mujer


In [11]:
# Este es el DF que variables que decidimos agrupar por encuesta y hay que partir las categorias:
df_grouped_part = df_or.groupby('encuesta')[['p_15','p_19','p_20','p_36',
                                            'p_43','p_45','p_49','p_66','p_67',
                                            'p_68','p_83','p_84','p_86']]

In [12]:
# Vamos a mirar p_19: año de nacimiento. Se crean grupos
df_or['p_19'].describe()
# max = 2018
# min = 1912
# p_19a: 0 - 10 (x <= 2020 and x >= 2010)
# p_19b: 11 - 19 (x <= 2009 and x >= 2001)
# p_19c: 20 - 35 (x <= 2000 and x >= 1985)
# p_19d: 36 - 65 (x <= 1984 and x >= 1955)
# p_19e: >66 (x < 1955)

p_19a = list(map((lambda x : x <= 2020 and x >= 2010), df_or['p_19']))
p_19b = list(map((lambda x : x <= 2009 and x >= 2001), df_or['p_19'])) 
p_19c = list(map((lambda x : x <= 2000 and x >= 1985), df_or['p_19']))
p_19d = list(map((lambda x : x <= 1984 and x >= 1955), df_or['p_19']))
p_19e = list(map((lambda x : x < 1955), df_or['p_19']))
enc = list(df_or['encuesta'])
df_ed = pd.DataFrame([])
df_ed['encuesta'] = enc
df_ed['p_19a'] = pd.DataFrame(p_19a)
df_ed['p_19b'] = pd.DataFrame(p_19b)
df_ed['p_19c'] = pd.DataFrame(p_19c)
df_ed['p_19d'] = pd.DataFrame(p_19d)
df_ed['p_19e'] = pd.DataFrame(p_19e)
df_ed.head(20)
df_ed.replace(to_replace=False,value=0)

df_edad = df_ed.groupby('encuesta')['p_19a'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_19a')
df_edadb = df_ed.groupby('encuesta')['p_19b'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_19b')
df_edadc = df_ed.groupby('encuesta')['p_19c'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_19c')
df_edadd = df_ed.groupby('encuesta')['p_19d'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_19d')
df_edade = df_ed.groupby('encuesta')['p_19e'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_19e')

In [13]:
df_agrup = pd.merge(df_edad, df_edadb, on='encuesta')
df_agrup = pd.merge(df_agrup, df_edadc, on='encuesta')
df_agrup = pd.merge(df_agrup, df_edadd, on='encuesta')
df_agrup = pd.merge(df_agrup, df_edade, on='encuesta')
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e
0,12016.0,0,0,2,3,1
1,12017.0,0,0,2,2,0
2,12018.0,1,1,2,1,0
3,22016.0,0,0,1,1,0
4,22017.0,0,2,3,1,0


In [14]:
#p_20 - estado civil actual
    #1	Soltero(a)
    #2	Casado(a) 
    #3	Viudo(a)
    #4	Separado o Divorciado(a)
    #5	No está casado y vive en pareja hace menos de 2 años
    #6	No está casado y vive en pareja hace 2 años o más
  
# p_20a-soltero [1,3,4]
# p_20b-pareja [2,5,6])

p_20a = list(map((lambda x : x==1 or x==3 or x==4), df_or['p_20']))
p_20b = list(map((lambda x : x==2 or x==5 or x==6), df_or['p_20']))

enc = list(df_or['encuesta'])
df_civil = pd.DataFrame([])
df_civil['encuesta'] = enc
df_civil['p_20a'] = pd.DataFrame(p_20a)
df_civil['p_20b'] = pd.DataFrame(p_20b)

df_civil.replace(to_replace=False,value=0)

df_soltero = df_civil.groupby('encuesta')['p_20a'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_20a')
df_pareja = df_civil.groupby('encuesta')['p_20b'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_20b')

df_agrup = pd.merge(df_agrup, df_soltero, on='encuesta')
df_agrup = pd.merge(df_agrup, df_pareja, on='encuesta')
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b
0,12016.0,0,0,2,3,1,6,0
1,12017.0,0,0,2,2,0,2,2
2,12018.0,1,1,2,1,0,5,0
3,22016.0,0,0,1,1,0,2,0
4,22017.0,0,2,3,1,0,6,0


In [15]:
#'p_36': Actualmente estudia? (asiste a salacuna, guarderia, preescolar, escuela, colegio, tecnico, tecnologico o universidad)
# 1	Si
# 2	No

df_estudia = df_or.groupby('encuesta')['p_36'].apply(lambda x: (x==1).sum()).reset_index(name='p_36a')
df_no_estudia = df_or.groupby('encuesta')['p_36'].apply(lambda x: (x==2).sum()).reset_index(name='p_36b')
df_agrup = pd.merge(df_agrup, df_estudia, on='encuesta')
df_agrup = pd.merge(df_agrup, df_no_estudia, on='encuesta')
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b
0,12016.0,0,0,2,3,1,6,0,2,4
1,12017.0,0,0,2,2,0,2,2,2,2
2,12018.0,1,1,2,1,0,5,0,1,4
3,22016.0,0,0,1,1,0,2,0,1,1
4,22017.0,0,2,3,1,0,6,0,4,2


In [16]:
#'p_43': Que medio de transporte utiliza predominantemente para dirigirse a su centro educativo?
# -99	No responde ¿?
# -88	No aplica
# -77	Otro.
# 1	Caminando
# 2	Bicicleta
# 3	Moto
# 4	Bus/Buseta/Ejecutivo
# 5	Transporte Informal
# 6	Taxi/Colectivo
# 7	Transporte Privado
# 8	Metro/Cable
# 9	Sistema integrado de transporte (SIT).
# 10	Caballo (bestia)
# 11	Lancha

# p43na: -99, -88 
# p43a: -77, 10, 11
# p43b: 4, 5, 6, 8, 9
# p43c: 3,7
# p43d: 1,2

p_43na = list(map((lambda x : x==-99 or x == -88), df_or['p_43']))
p_43a = list(map((lambda x : x== -77 or x == 10 or x == 11), df_or['p_43'])) 
p_43b = list(map((lambda x : x==4 or x == 5 or x==6 or x == 8 or x==9), df_or['p_43']))
p_43c = list(map((lambda x : x==3 or x == 7), df_or['p_43']))
p_43d = list(map((lambda x : x==1 or x == 2), df_or['p_43']))

enc = list(df_or['encuesta'])
df_t_ed = pd.DataFrame([])

df_t_ed['encuesta'] = enc
df_t_ed['p_43na'] = pd.DataFrame(p_43na)
df_t_ed['p_43a'] = pd.DataFrame(p_43a)
df_t_ed['p_43b'] = pd.DataFrame(p_43b)
df_t_ed['p_43c'] = pd.DataFrame(p_43c)
df_t_ed['p_43d'] = pd.DataFrame(p_43d)
df_t_ed.head(20)
df_t_ed.replace(to_replace=False,value=0)

df_ted = df_t_ed.groupby('encuesta')['p_43na'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_43na')
df_tedb = df_t_ed.groupby('encuesta')['p_43a'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_43a')
df_tedc = df_t_ed.groupby('encuesta')['p_43b'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_43b')
df_tedd = df_t_ed.groupby('encuesta')['p_43c'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_43c')
df_tede = df_t_ed.groupby('encuesta')['p_43d'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_43d')

df_agrup = pd.merge(df_agrup, df_ted, on='encuesta')
df_agrup = pd.merge(df_agrup, df_tedb, on='encuesta')
df_agrup = pd.merge(df_agrup, df_tedc, on='encuesta')
df_agrup = pd.merge(df_agrup, df_tedd, on='encuesta')
df_agrup = pd.merge(df_agrup, df_tede, on='encuesta')
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2


In [17]:
# 'p_45': ultimo NIVEL de estudio aprobado (titulo):
  # 0	Ninguno, 
  # -99	No responde
  # -98	No sabe
  # 1 Salacuna
  #  2	Primaria (1º a 5º)
  # 3	Secundaria (6º a 9º)
  # 4	Media académica o Normalista (10º a 13º), 5	Media Técnica (dos años), 6	Tecnológico 
  # 7	Universidad 
  # 8	Especializacion, 9	Maestria, 10	Doctorado (5 años)
  
# p45a: -99 (no responde), -88 (no aplica), 0 (ninguno), 1 (salacuna) 
# p45b: 2 (primaria)
# p45c: 3 (secundaria)
# p45d: 4 Media académica o Normalista (10º a 13º) , 5 Media tecnica 2 años, 6 Tecnologico
# p45e: 7 Universidad
# p45f: 8 (Especializacion), 9 (Maestria), 10 (PhD 5 años) 

p_45a = list(map((lambda x : x==-99 or x == -88 or x==0 or x==1), df_or['p_45']))
p_45b = list(map((lambda x : x==2), df_or['p_45'])) 
p_45c = list(map((lambda x : x==3), df_or['p_45']))
p_45d = list(map((lambda x : x==4 or x == 5 or x==6), df_or['p_45']))
p_45e = list(map((lambda x : x==7), df_or['p_45']))
p_45f = list(map((lambda x :  x==8 or x==9 or x==10), df_or['p_45']))

enc = list(df_or['encuesta'])
df_ult_niv = pd.DataFrame([])

df_ult_niv['encuesta'] = enc
df_ult_niv['p_45a'] = pd.DataFrame(p_45a)
df_ult_niv['p_45b'] = pd.DataFrame(p_45b)
df_ult_niv['p_45c'] = pd.DataFrame(p_45c)
df_ult_niv['p_45d'] = pd.DataFrame(p_45d)
df_ult_niv['p_45e'] = pd.DataFrame(p_45e)
df_ult_niv['p_45f'] = pd.DataFrame(p_45f)
df_ult_niv.head(20)
df_ult_niv.replace(to_replace=False,value=0)

df_niva = df_ult_niv.groupby('encuesta')['p_45a'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_45a')
df_nivb = df_ult_niv.groupby('encuesta')['p_45b'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_45b')
df_nivc = df_ult_niv.groupby('encuesta')['p_45c'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_45c')
df_nivd = df_ult_niv.groupby('encuesta')['p_45d'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_45d')
df_nive = df_ult_niv.groupby('encuesta')['p_45e'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_45e')
df_nivf = df_ult_niv.groupby('encuesta')['p_45f'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_45f')

df_agrup = pd.merge(df_agrup, df_niva, on='encuesta')
df_agrup = pd.merge(df_agrup, df_nivb, on='encuesta')
df_agrup = pd.merge(df_agrup, df_nivc, on='encuesta')
df_agrup = pd.merge(df_agrup, df_nivd, on='encuesta')
df_agrup = pd.merge(df_agrup, df_nive, on='encuesta')
df_agrup = pd.merge(df_agrup, df_nivf, on='encuesta')
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0


In [18]:
# 'p_49': El carácter del ESTABLECIMIENTO EDUCATIVO (escuela, colegio, universidad u otra institucion) en donde estudia o estudió el curso o grado, en el último año, es:
# -99	No responde,
# -98	No sabe
# -88	No aplica
# 1	Público
# 2	Privado

# p_49a: -99 no responde, -98 no sabe, -88 no aplica
# p_49b: 1 publico
# p_49c: 2 privado

p_49a = list(map((lambda x : x==-99 or x == -98 or x==-88), df_or['p_49']))
p_49b = list(map((lambda x : x==1), df_or['p_49'])) 
p_49c = list(map((lambda x : x==2), df_or['p_49']))

enc = list(df_or['encuesta'])
df_pub_priv = pd.DataFrame([])

df_pub_priv['encuesta'] = enc
df_pub_priv['p_49a'] = pd.DataFrame(p_49a)
df_pub_priv['p_49b'] = pd.DataFrame(p_49b)
df_pub_priv['p_49c'] = pd.DataFrame(p_49c)

df_pa = df_pub_priv.groupby('encuesta')['p_49a'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_49a')
df_pb = df_pub_priv.groupby('encuesta')['p_49b'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_49b')
df_pc = df_pub_priv.groupby('encuesta')['p_49c'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_49c')

df_agrup = pd.merge(df_agrup, df_pa, on='encuesta')
df_agrup = pd.merge(df_agrup, df_pb, on='encuesta')
df_agrup = pd.merge(df_agrup, df_pc, on='encuesta')

df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0,0,5,1
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0,0,4,0
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0,1,4,0
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,2
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0,0,6,0


In [19]:
# 'p_66': Tipo de afiliacion al sistema de seguridad social en salud
# -99	No responde
# -98	No sabe
# 7	No está afiliado y no está encuestado en el SISBEN

# 1	Es contributivo cotizante. Tiene EPS.
# 4	Régimen especial: (FFAA)

# 2	Beneficiario del régimen contributivo.
# 5	Beneficiario del Régimen especial

# 3	Subsidiado
# 6	No está afiliado y está encuestado en el SISBEN

# p_66a: -99 no responde, -98 no sabe, 7 no esta afiliado y no está encuestado SISBEN
# p_66b: 1 Es contributivo cotizante. Tiene EPS, 4 Régimen especial: (FFAA)
# p_66c: 2	Beneficiario del régimen contributivo, 5	Beneficiario del Régimen especial
# p_66d: 3	Subsidiado, 6	No está afiliado y está encuestado en el SISBEN

p_66a = list(map((lambda x : x== -99 or x == -98 or x == 7), df_or['p_66'])) 
p_66b = list(map((lambda x : x==1 or x == 4), df_or['p_66']))
p_66c = list(map((lambda x : x==2 or x == 5), df_or['p_66']))
p_66d = list(map((lambda x : x==3 or x == 6), df_or['p_66']))

enc = list(df_or['encuesta'])
df_eps = pd.DataFrame([])

df_eps['encuesta'] = enc
df_eps['p_66a'] = pd.DataFrame(p_66a)
df_eps['p_66b'] = pd.DataFrame(p_66b)
df_eps['p_66c'] = pd.DataFrame(p_66c)
df_eps['p_66d'] = pd.DataFrame(p_66d)
df_eps.replace(to_replace=False,value=0)

df_epsa = df_eps.groupby('encuesta')['p_66a'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_66a')
df_epsb = df_eps.groupby('encuesta')['p_66b'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_66b')
df_epsc = df_eps.groupby('encuesta')['p_66c'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_66c')
df_epsd = df_eps.groupby('encuesta')['p_66d'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_66d')


df_agrup = pd.merge(df_agrup, df_epsa, on='encuesta')
df_agrup = pd.merge(df_agrup, df_epsb, on='encuesta')
df_agrup = pd.merge(df_agrup, df_epsc, on='encuesta')
df_agrup = pd.merge(df_agrup, df_epsd, on='encuesta')
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c,p_66a,p_66b,p_66c,p_66d
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0,0,5,1,0,3,3,0
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0,0,4,0,0,0,0,4
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0,1,4,0,0,0,0,5
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,2,0,1,1,0
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0,0,6,0,0,0,6,0


In [20]:
# 'p_67': Afiliacion al sistema de pensiones
# -88	No aplica: p_67a
# 1	Si está afiliado: p_67b
# 2	No está afiliado: p_67c

df_na = df_or.groupby('encuesta')['p_67'].apply(lambda x: (x==-88).sum()).reset_index(name='p_67a')
df_af = df_or.groupby('encuesta')['p_67'].apply(lambda x: (x==1).sum()).reset_index(name='p_67b')
df_naf = df_or.groupby('encuesta')['p_67'].apply(lambda x: (x==2).sum()).reset_index(name='p_67c')
 
df_agrup = pd.merge(df_agrup, df_na, on='encuesta')
df_agrup = pd.merge(df_agrup, df_af, on='encuesta') 
df_agrup = pd.merge(df_agrup, df_naf, on='encuesta') 
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c,p_66a,p_66b,p_66c,p_66d,p_67a,p_67b,p_67c
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0,0,5,1,0,3,3,0,0,3,3
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0,0,4,0,0,0,0,4,0,0,4
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0,1,4,0,0,0,0,5,2,0,3
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,2,0,1,1,0,0,1,1
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0,0,6,0,0,0,6,0,0,0,6


In [21]:
# 'p_68': Afiliacion a una Administradora de Riesgos Laborales (ARL)
# -88	No aplica: p_68a
# 1	Si está afiliado: p_68b
# 2	No está afiliado: p_68c

df_nar = df_or.groupby('encuesta')['p_68'].apply(lambda x: (x==-88).sum()).reset_index(name='p_68a')
df_afr = df_or.groupby('encuesta')['p_68'].apply(lambda x: (x==1).sum()).reset_index(name='p_68b')
df_nafr = df_or.groupby('encuesta')['p_68'].apply(lambda x: (x==2).sum()).reset_index(name='p_68c')
 
df_agrup = pd.merge(df_agrup, df_nar, on='encuesta')
df_agrup = pd.merge(df_agrup, df_afr, on='encuesta') 
df_agrup = pd.merge(df_agrup, df_nafr, on='encuesta') 
df_agrup.head()


,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c,p_66a,p_66b,p_66c,p_66d,p_67a,p_67b,p_67c,p_68a,p_68b,p_68c
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0,0,5,1,0,3,3,0,0,3,3,0,3,3
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0,0,4,0,0,0,0,4,0,0,4,0,0,4
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0,1,4,0,0,0,0,5,2,0,3,2,0,3
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,2,0,1,1,0,0,1,1,0,1,1
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0,0,6,0,0,0,6,0,0,0,6,0,0,6


In [22]:
#'p_86': En este trabajo es:
    # -99 No responde
    # -98 No sabe
    # -88 No aplica 
    # -77 Otro
    # 1 Obrero o empleado de empresa particular
    # 2 Obrero o empleado del Gobierno 
    # 3 Empleado(a) doméstico(a) 
    # 4 Trabajador por cuenta propia formal
    # 5 Patrón o empleador
    # 6 Trabajador familiar sin remuneración (cuida familiares y otros)
    # 7 Trabajador sin remuneración en empresas o negocios de otros hogares
    # 8 Jornalero o peón
    # 9 Contratista 
    # 10 Trabajador informal   
    # 11 Trabajador por días 

# p_86a: 0 [-99,-98,-88]
# p_86b: 1 [1, 2] 
# p_86c: 2 [3] 
# p_86d: 3 [4,9] 
# p_86e: 4 [5] 
# p_86f: 5 [6,7] 
# p_86g: 6 [8] 
# p_86h: 7 [10,11]
# p_86i: 8 [-77]

p_86a = list(map((lambda x : x==-99 or x==-98 or x==-88), df_or['p_86']))
p_86b = list(map((lambda x : x==1 or x==2 ), df_or['p_86']))
p_86c = list(map((lambda x : x==3), df_or['p_86']))
p_86d = list(map((lambda x : x==4 or x==9 ), df_or['p_86']))
p_86e = list(map((lambda x : x==5), df_or['p_86']))
p_86f = list(map((lambda x : x==6 or x==7 ), df_or['p_86']))
p_86g = list(map((lambda x : x==8), df_or['p_86']))
p_86h = list(map((lambda x : x==10 or x==11 ), df_or['p_86']))
p_86i = list(map((lambda x : x==-77), df_or['p_86']))

enc = list(df_or['encuesta'])
df_rol_trab = pd.DataFrame([])
df_rol_trab['encuesta'] = enc
df_rol_trab['p_86a'] = pd.DataFrame(p_86a)
df_rol_trab['p_86b'] = pd.DataFrame(p_86b)
df_rol_trab['p_86c'] = pd.DataFrame(p_86c)
df_rol_trab['p_86d'] = pd.DataFrame(p_86d)
df_rol_trab['p_86e'] = pd.DataFrame(p_86e)
df_rol_trab['p_86f'] = pd.DataFrame(p_86f)
df_rol_trab['p_86g'] = pd.DataFrame(p_86g)
df_rol_trab['p_86h'] = pd.DataFrame(p_86h)
df_rol_trab['p_86i'] = pd.DataFrame(p_86i)

df_rol_trab.replace(to_replace=False,value=0)

df_rol_trab_a = df_rol_trab.groupby('encuesta')['p_86a'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_86a')
df_rol_trab_b = df_rol_trab.groupby('encuesta')['p_86b'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_86b')
df_rol_trab_c = df_rol_trab.groupby('encuesta')['p_86c'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_86c')
df_rol_trab_d = df_rol_trab.groupby('encuesta')['p_86d'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_86d')
df_rol_trab_e = df_rol_trab.groupby('encuesta')['p_86e'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_86e')
df_rol_trab_f = df_rol_trab.groupby('encuesta')['p_86f'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_86f')
df_rol_trab_g = df_rol_trab.groupby('encuesta')['p_86g'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_86g')
df_rol_trab_h = df_rol_trab.groupby('encuesta')['p_86h'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_86h')
df_rol_trab_i = df_rol_trab.groupby('encuesta')['p_86i'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_86i')

df_agrup = pd.merge(df_agrup, df_rol_trab_a, on='encuesta')
df_agrup = pd.merge(df_agrup, df_rol_trab_b, on='encuesta')
df_agrup = pd.merge(df_agrup, df_rol_trab_c, on='encuesta')
df_agrup = pd.merge(df_agrup, df_rol_trab_d, on='encuesta')
df_agrup = pd.merge(df_agrup, df_rol_trab_e, on='encuesta')
df_agrup = pd.merge(df_agrup, df_rol_trab_f, on='encuesta')
df_agrup = pd.merge(df_agrup, df_rol_trab_g, on='encuesta')
df_agrup = pd.merge(df_agrup, df_rol_trab_h, on='encuesta')
df_agrup = pd.merge(df_agrup, df_rol_trab_i, on='encuesta')
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c,p_66a,p_66b,p_66c,p_66d,p_67a,p_67b,p_67c,p_68a,p_68b,p_68c,p_86a,p_86b,p_86c,p_86d,p_86e,p_86f,p_86g,p_86h,p_86i
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0,0,5,1,0,3,3,0,0,3,3,0,3,3,5,1,0,0,0,0,0,0,0
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0,0,4,0,0,0,0,4,0,0,4,0,0,4,4,0,0,0,0,0,0,0,0
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0,1,4,0,0,0,0,5,2,0,3,2,0,3,4,1,0,0,0,0,0,0,0
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,2,0,1,1,0,0,1,1,0,1,1,1,0,0,1,0,0,0,0,0
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0,0,6,0,0,0,6,0,0,0,6,0,0,6,5,0,0,0,0,0,0,1,0


In [23]:
# 'p_84': Cuanto tiempo tardan en llegar al sitio de trabajo?
#     -88	No aplica
#     1	De 0 a 20 minutos
#     2	De 21 a 40 minutos
#     3	De 41 a 60 minutos
#     4	De 61 a 80 minutos
#|    5	Más de 81 minutos

#   0 - [-88]
#   1 - [1,2]
#   2 - [3,4]
#   3 - [5]

p_84a = list(map((lambda x : x==-88), df_or['p_84']))
p_84b = list(map((lambda x : x==1 or x==2), df_or['p_84']))
p_84c = list(map((lambda x : x==3 or x==4), df_or['p_84']))
p_84d = list(map((lambda x : x==5 ), df_or['p_84']))

enc = list(df_or['encuesta'])
df_tiempo_trabajo = pd.DataFrame([])
df_tiempo_trabajo['encuesta'] = enc
df_tiempo_trabajo['p_84a'] = pd.DataFrame(p_84a)
df_tiempo_trabajo['p_84b'] = pd.DataFrame(p_84b)
df_tiempo_trabajo['p_84c'] = pd.DataFrame(p_84c)
df_tiempo_trabajo['p_84d'] = pd.DataFrame(p_84d)

df_tiempo_trabajo.replace(to_replace=False,value=0)

df_tiempo_trabajo_a = df_tiempo_trabajo.groupby('encuesta')['p_84a'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_84a')
df_tiempo_trabajo_b = df_tiempo_trabajo.groupby('encuesta')['p_84b'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_84b')
df_tiempo_trabajo_c = df_tiempo_trabajo.groupby('encuesta')['p_84c'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_84c')
df_tiempo_trabajo_d = df_tiempo_trabajo.groupby('encuesta')['p_84d'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_84d')

df_agrup = pd.merge(df_agrup, df_tiempo_trabajo_a, on='encuesta')
df_agrup = pd.merge(df_agrup, df_tiempo_trabajo_b, on='encuesta')
df_agrup = pd.merge(df_agrup, df_tiempo_trabajo_c, on='encuesta')
df_agrup = pd.merge(df_agrup, df_tiempo_trabajo_d, on='encuesta')
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c,p_66a,p_66b,p_66c,p_66d,p_67a,p_67b,p_67c,p_68a,p_68b,p_68c,p_86a,p_86b,p_86c,p_86d,p_86e,p_86f,p_86g,p_86h,p_86i,p_84a,p_84b,p_84c,p_84d
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0,0,5,1,0,3,3,0,0,3,3,0,3,3,5,1,0,0,0,0,0,0,0,5,1,0,0
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0,0,4,0,0,0,0,4,0,0,4,0,0,4,4,0,0,0,0,0,0,0,0,4,0,0,0
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0,1,4,0,0,0,0,5,2,0,3,2,0,3,4,1,0,0,0,0,0,0,0,4,1,0,0
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,2,0,1,1,0,0,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,0,0
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0,0,6,0,0,0,6,0,0,0,6,0,0,6,5,0,0,0,0,0,0,1,0,5,1,0,0


In [24]:
# 'p_83': Cual es el Medio de transporte utilizado predominante para dirigirse al sitio de su empleo principal
#    - -99	No responde
#    - -88	No aplica
#    - -77	Otro.
#    - 1	Caminando
#    - 2	Bicicleta
#    - 3	Moto
#    - 4	Bus/Buseta/Ejecutivo
#    - 5	Transporte Informal
#    - 6	Taxi o colectivo
#    - 7	Transporte particular
#    - 8	Metro - cable
#    - 9	Sistema Integrado de Transporte (SIT)
#    - 10	Ferrocarril
#    - 11	Aéreo
#    - 12	Caballo (bestia)
#    - 13	Lancha

#   p_83a - [-99, -88] -> Otro
#   p_83b -[-77, 10, 12, 13] ->  transp. publico
#   p_83c -[4, 5, 6, 8, 9] -> trasp. privado
#   p_83d -[3,7] -> trasp. privado 
#   p_83e -[1,2] -> no motor
#   p_83f -[11] -> aereo 

p_83a = list(map((lambda x : x==-99 or x==-88), df_or['p_83']))
p_83b = list(map((lambda x : x==-77 or x==10 or x==12 or x==13), df_or['p_83']))
p_83c = list(map((lambda x : x==4 or x==5 or x==6 or x==8 or x==9 ), df_or['p_83']))
p_83d = list(map((lambda x : x==3 or x==7 ), df_or['p_83']))
p_83e = list(map((lambda x : x==1 or x==2), df_or['p_83']))
p_83f = list(map((lambda x : x==11 ), df_or['p_83']))

enc = list(df_or['encuesta'])
df_med_tran = pd.DataFrame([])
df_med_tran['encuesta'] = enc
df_med_tran['p_83a'] = pd.DataFrame(p_83a)
df_med_tran['p_83b'] = pd.DataFrame(p_83b)
df_med_tran['p_83c'] = pd.DataFrame(p_83c)
df_med_tran['p_83d'] = pd.DataFrame(p_83d)
df_med_tran['p_83e'] = pd.DataFrame(p_83e)
df_med_tran['p_83f'] = pd.DataFrame(p_83f)

df_med_tran.replace(to_replace=False,value=0)

df_med_tran_a = df_med_tran.groupby('encuesta')['p_83a'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_83a')
df_med_tran_b = df_med_tran.groupby('encuesta')['p_83b'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_83b')
df_med_tran_c = df_med_tran.groupby('encuesta')['p_83c'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_83c')
df_med_tran_d = df_med_tran.groupby('encuesta')['p_83d'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_83d')
df_med_tran_e = df_med_tran.groupby('encuesta')['p_83e'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_83e')
df_med_tran_f = df_med_tran.groupby('encuesta')['p_83f'].apply(lambda x: (x==1.0).sum()).reset_index(name='p_83f')

df_agrup = pd.merge(df_agrup, df_med_tran_a, on='encuesta')
df_agrup = pd.merge(df_agrup, df_med_tran_b, on='encuesta')
df_agrup = pd.merge(df_agrup, df_med_tran_c, on='encuesta')
df_agrup = pd.merge(df_agrup, df_med_tran_d, on='encuesta')
df_agrup = pd.merge(df_agrup, df_med_tran_e, on='encuesta')
df_agrup = pd.merge(df_agrup, df_med_tran_f, on='encuesta')
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c,p_66a,p_66b,p_66c,p_66d,p_67a,p_67b,p_67c,p_68a,p_68b,p_68c,p_86a,p_86b,p_86c,p_86d,p_86e,p_86f,p_86g,p_86h,p_86i,p_84a,p_84b,p_84c,p_84d,p_83a,p_83b,p_83c,p_83d,p_83e,p_83f
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0,0,5,1,0,3,3,0,0,3,3,0,3,3,5,1,0,0,0,0,0,0,0,5,1,0,0,5,0,0,1,0,0
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0,0,4,0,0,0,0,4,0,0,4,0,0,4,4,0,0,0,0,0,0,0,0,4,0,0,0,4,0,0,0,0,0
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0,1,4,0,0,0,0,5,2,0,3,2,0,3,4,1,0,0,0,0,0,0,0,4,1,0,0,4,0,1,0,0,0
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,2,0,1,1,0,0,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0,0,6,0,0,0,6,0,0,0,6,0,0,6,5,0,0,0,0,0,0,1,0,5,1,0,0,5,0,0,0,1,0


In [25]:
# Partición del sexo, p_15a: # de hombres por hogar y p_15b: # de mujeres por hogar
#df_sexo = df.groupby(['encuesta', 'p_15']).size().reset_index(name='count') #Divide los datos dependiendo de los posibles valores de p_15
#En las siguientes lineas dividimos los valores del sexo masculino y femenino por cada encuesta.
df_sexo_m = df_or.groupby('encuesta')['p_15'].apply(lambda x: (x==1).sum()).reset_index(name='p_15a')
df_sexo_f = df_or.groupby('encuesta')['p_15'].apply(lambda x: (x==2).sum()).reset_index(name='p_15b')

# Se le agrega pregunta 15a y 15b al df que tiene TODAS las variables que vamos a usar.... 
#En estas lineas agregamos los df de los hombres y mujeres de cada encuesta.
df_agrup = pd.merge(df_agrup, df_sexo_m, on='encuesta')
df_agrup = pd.merge(df_agrup, df_sexo_f, on='encuesta')

In [26]:
#df_agrup.info()

In [27]:
preg7 = df_or.groupby('encuesta')['p_7'].first()
preg7.head()

encuesta
12016.0    1108
12017.0    0101
12018.0    0303
22016.0    1104
22017.0    0101
Name: p_7, dtype: object

In [28]:
df_agrup = pd.merge(df_agrup, preg7, on='encuesta')

In [29]:
df_agrup.head()

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c,p_66a,p_66b,p_66c,p_66d,p_67a,p_67b,p_67c,p_68a,p_68b,p_68c,p_86a,p_86b,p_86c,p_86d,p_86e,p_86f,p_86g,p_86h,p_86i,p_84a,p_84b,p_84c,p_84d,p_83a,p_83b,p_83c,p_83d,p_83e,p_83f,p_15a,p_15b,p_7
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0,0,5,1,0,3,3,0,0,3,3,0,3,3,5,1,0,0,0,0,0,0,0,5,1,0,0,5,0,0,1,0,0,0,6,1108
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0,0,4,0,0,0,0,4,0,0,4,0,0,4,4,0,0,0,0,0,0,0,0,4,0,0,0,4,0,0,0,0,0,3,1,0101
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0,1,4,0,0,0,0,5,2,0,3,2,0,3,4,1,0,0,0,0,0,0,0,4,1,0,0,4,0,1,0,0,0,2,3,0303
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,2,0,1,1,0,0,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,1,1104
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0,0,6,0,0,0,6,0,0,0,6,0,0,6,5,0,0,0,0,0,0,1,0,5,1,0,0,5,0,0,0,1,0,3,3,0101


In [30]:
df_y = df_or.groupby(by='encuesta')['año'].max()
df_y

encuesta
12016.0        2016
12017.0        2017
12018.0        2018
22016.0        2016
22017.0        2017
               ... 
168852018.0    2018
168982018.0    2018
169452018.0    2018
178842018.0    2018
191862018.0    2018
Name: año, Length: 27612, dtype: int64

In [31]:
df_agrup = pd.merge(df_agrup, df_y, on='encuesta')

In [32]:
df_agrup

,encuesta,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43na,p_43a,p_43b,p_43c,p_43d,p_45a,p_45b,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c,p_66a,p_66b,p_66c,p_66d,p_67a,p_67b,p_67c,p_68a,p_68b,p_68c,p_86a,p_86b,p_86c,p_86d,p_86e,p_86f,p_86g,p_86h,p_86i,p_84a,p_84b,p_84c,p_84d,p_83a,p_83b,p_83c,p_83d,p_83e,p_83f,p_15a,p_15b,p_7,año
0,12016.0,0,0,2,3,1,6,0,2,4,4,0,2,0,0,0,1,0,4,1,0,0,5,1,0,3,3,0,0,3,3,0,3,3,5,1,0,0,0,0,0,0,0,5,1,0,0,5,0,0,1,0,0,0,6,1108,2016
1,12017.0,0,0,2,2,0,2,2,2,2,2,0,2,0,0,1,0,0,3,0,0,0,4,0,0,0,0,4,0,0,4,0,0,4,4,0,0,0,0,0,0,0,0,4,0,0,0,4,0,0,0,0,0,3,1,0101,2017
2,12018.0,1,1,2,1,0,5,0,1,4,4,0,0,0,1,2,0,3,0,0,0,1,4,0,0,0,0,5,2,0,3,2,0,3,4,1,0,0,0,0,0,0,0,4,1,0,0,4,0,1,0,0,0,2,3,0303,2018
3,22016.0,0,0,1,1,0,2,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,2,0,1,1,0,0,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,1,1104,2016
4,22017.0,0,2,3,1,0,6,0,4,2,2,0,1,1,2,1,3,0,2,0,0,0,6,0,0,0,6,0,0,0,6,0,0,6,5,0,0,0,0,0,0,1,0,5,1,0,0,5,0,0,0,1,0,3,3,0101,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27607,168852018.0,1,0,0,2,1,2,2,1,3,3,0,0,1,0,1,0,1,2,0,0,0,3,1,0,3,1,0,1,2,1,1,2,1,3,1,0,0,0,0,0,0,0,3,1,0,0,3,0,0,1,0,0,2,2,1301,2018
27608,168982018.0,0,1,1,1,2,3,2,1,4,4,0,1,0,0,0,1,0,3,1,0,0,3,2,0,4,1,0,1,2,2,1,2,2,4,1,0,0,0,0,0,0,0,4,0,1,0,4,0,0,1,0,0,2,3,1301,2018
27609,169452018.0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1408,2018
27610,178842018.0,1,0,1,0,1,3,0,1,2,2,0,0,1,0,1,0,0,1,1,0,0,2,1,0,2,1,0,1,1,1,1,1,1,3,0,0,0,0,0,0,0,0,2,0,0,1,2,0,1,0,0,0,2,1,1609,2018


In [33]:
df_final = pd.merge(df_grouped_max, df_agrup, on='encuesta')
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27612 entries, 0 to 27611
Columns: 134 entries, encuesta to año
dtypes: float64(2), int64(131), object(1)
memory usage: 28.4+ MB


In [34]:
df_final.head()

,encuesta,p_5,p_6,p_10,p_12,p_44,p_88,p_89,p_90,p_91,p_92,p_93,p_112,p_113,p_114,p_115,p_125,p_126,p_146,p_147,p_148,p_157,p_158,p_162,p_165,p_171,p_174,p_178,p_184,p_211,p_212,p_213,p_214,p_227,p_232,p_233,p_234,p_235,p_236,p_237,...,p_45c,p_45d,p_45e,p_45f,p_49a,p_49b,p_49c,p_66a,p_66b,p_66c,p_66d,p_67a,p_67b,p_67c,p_68a,p_68b,p_68c,p_86a,p_86b,p_86c,p_86d,p_86e,p_86f,p_86g,p_86h,p_86i,p_84a,p_84b,p_84c,p_84d,p_83a,p_83b,p_83c,p_83d,p_83e,p_83f,p_15a,p_15b,p_7,año
0,12016.0,1,11,5,6,1,-88,-88,-88,-88,-88,999,-88,-88,-88,-88,-88,-88,5,10,4,9,1,1,1,1,1,1,6,24,1,0,0,-88,-99,-99,-99,-99,-99,-99,...,0,4,1,0,0,5,1,0,3,3,0,0,3,3,0,3,3,5,1,0,0,0,0,0,0,0,5,1,0,0,5,0,0,1,0,0,0,6,1108,2016
1,12017.0,1,1,2,4,4,-88,-88,-88,-88,-88,-88,600000,0,0,0,0,0,4,7,4,3,1,1,1,1,1,1,6,9,0,1,0,-88,400000,0,120000,100000,0,50000,...,0,3,0,0,0,4,0,0,0,0,4,0,0,4,0,0,4,4,0,0,0,0,0,0,0,0,4,0,0,0,4,0,0,0,0,0,3,1,0101,2017
2,12018.0,1,3,2,5,1,0,0,-88,0,0,0,0,0,0,0,0,0,4,10,4,4,1,1,1,1,1,2,6,7,0,0,0,-88,1200000,0,0,400000,0,0,...,3,0,0,0,1,4,0,0,0,0,5,2,0,3,2,0,3,4,1,0,0,0,0,0,0,0,4,1,0,0,4,0,1,0,0,0,2,3,0303,2018
3,22016.0,1,11,4,2,1,-88,-88,300000,0,0,0,-88,-88,-88,-88,2000000,0,4,10,4,4,1,1,1,1,1,1,6,13,0,0,1,900000,2500000,390000,0,600000,0,0,...,1,1,0,0,0,0,2,0,1,1,0,0,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,1,1104,2016
4,22017.0,1,1,2,6,2,0,0,0,0,0,0,-88,0,0,0,-88,-88,5,10,4,3,1,1,2,1,1,2,4,8,0,1,0,-88,400000,0,0,240000,0,45000,...,0,2,0,0,0,6,0,0,0,6,0,0,0,6,0,0,6,5,0,0,0,0,0,0,1,0,5,1,0,0,5,0,0,0,1,0,3,3,0101,2017


In [35]:
df_final[['p_7']] = df_final[['p_7']].replace('AE2', '9014')
df_final[['p_7']] = df_final[['p_7']].replace('AUC1', '9015')
df_final[['p_7']] = df_final[['p_7']].replace('AE5', '9016')
df_final[['p_7']] = df_final[['p_7']].replace('AE6', '9017')
df_final[['p_7']] = df_final[['p_7']].replace('AE7', '9018')
df_final[['p_7']] = df_final[['p_7']].replace('AUC2', '9019')

Luego de realizar un análisis exploratorio de datos, concluimos que algunas de las preguntas tenían mucha información vacía y fueron excluidas.

In [36]:
#Filtrar preguntas y cambiar orden de campos
df_final = df_final[['año',
'encuesta',
'p_5',
'p_6',
'p_7',
'p_10',
'p_12',
'p_15a',
'p_15b',
'p_19a',
'p_19b',
'p_19c',
'p_19d',
'p_19e',
'p_20a',
'p_20b',
'p_36a',
'p_36b',
'p_43a',
'p_43b',
'p_43c',
'p_43d',
'p_44',
'p_45b',
'p_45c',
'p_45d',
'p_45e',
'p_45f',
'p_49b',
'p_49c',
'p_66b',
'p_66c',
'p_66d',
'p_67b',
'p_67c',
'p_68b',
'p_68c',
'p_83b',
'p_83c',
'p_83d',
'p_83e',
'p_83f',
'p_84b',
'p_84c',
'p_84d',
'p_86a',
'p_86b',
'p_86c',
'p_86d',
'p_86e',
'p_86f',
'p_86g',
'p_86h',
'p_86i',
'p_112',
'p_113',
'p_114',
'p_125',
'p_126',
'p_146',
'p_147',
'p_148',
'p_157',
'p_158',
'p_162',
'p_165',
'p_171',
'p_174',
'p_178',
'p_184',
'p_211',
'p_212',
'p_213',
'p_214',
'p_227',
'p_232',
'p_244',
'p_258',
'p_259',
'p_260',
'p_281',
'p_282',
'p_283',
'p_284',
'p_285',
'p_286',
'p_312',
'p_313',
'p_314',
'p_315',
'p_316',
'p_317',
'p_318',
'p_319',
'p_320',
'p_321',
'p_322',
'p_323'
]]

In [37]:
df_final.head()

,año,encuesta,p_5,p_6,p_7,p_10,p_12,p_15a,p_15b,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43a,p_43b,p_43c,p_43d,p_44,p_45b,p_45c,p_45d,p_45e,p_45f,p_49b,p_49c,p_66b,p_66c,p_66d,p_67b,p_67c,p_68b,p_68c,p_83b,p_83c,p_83d,...,p_126,p_146,p_147,p_148,p_157,p_158,p_162,p_165,p_171,p_174,p_178,p_184,p_211,p_212,p_213,p_214,p_227,p_232,p_244,p_258,p_259,p_260,p_281,p_282,p_283,p_284,p_285,p_286,p_312,p_313,p_314,p_315,p_316,p_317,p_318,p_319,p_320,p_321,p_322,p_323
0,2016,12016.0,1,11,1108,5,6,0,6,0,0,2,3,1,6,0,2,4,0,2,0,0,1,1,0,4,1,0,5,1,3,3,0,3,3,3,3,0,0,1,...,-88,5,10,4,9,1,1,1,1,1,1,6,24,1,0,0,-88,-99,0,3,4,3,4,3,2,14,-88,5,3,3,3,4,4,4,4,4,3,4,3,4
1,2017,12017.0,1,1,0101,2,4,3,1,0,0,2,2,0,2,2,2,2,0,2,0,0,4,0,0,3,0,0,4,0,0,0,4,0,4,0,4,0,0,0,...,0,4,7,4,3,1,1,1,1,1,1,6,9,0,1,0,-88,400000,590000,1,3,3,4,1,2,1,7,19,1,1,2,2,1,2,3,3,2,3,3,3
2,2018,12018.0,1,3,0303,2,5,2,3,1,1,2,1,0,5,0,1,4,0,0,0,1,1,0,3,0,0,0,4,0,0,0,5,0,3,0,3,0,1,0,...,0,4,10,4,4,1,1,1,1,1,2,6,7,0,0,0,-88,1200000,740000,2,4,3,4,5,2,7,14,19,4,3,4,4,4,5,4,4,4,4,4,4
3,2016,22016.0,1,11,1104,4,2,1,1,0,0,1,1,0,2,0,1,1,0,0,0,1,1,0,1,1,0,0,0,2,1,1,0,1,1,1,1,0,0,0,...,0,4,10,4,4,1,1,1,1,1,1,6,13,0,0,1,900000,2500000,1746200,1,4,3,2,3,2,7,1,19,2,1,1,1,1,5,3,3,1,5,4,4
4,2017,22017.0,1,1,0101,2,6,3,3,0,2,3,1,0,6,0,4,2,0,1,1,2,2,3,0,2,0,0,6,0,0,6,0,0,6,0,6,0,0,0,...,-88,5,10,4,3,1,1,2,1,1,2,4,8,0,1,0,-88,400000,375000,-98,-98,1,4,3,2,7,14,1,3,2,3,4,4,2,2,4,3,4,4,3


Hacer cálculos para actualizar IPC por años en campos de ingresos y gastos

In [38]:
#Junio 14
df = df_final
#Cálculo del ingreso per-cápita por hogar a precio del 2018
df['p_244_c1'] = np.where(df['año'] == 2016, ((df['p_244']*(1 + 0.0575)*(1 + 0.0409))/df['p_12']),0)
df['p_244_c2'] = np.where(df['año'] == 2017, ((df['p_244']*(1 + 0.0409))/df['p_12']),0)
df['p_244_c3'] = np.where(df['año'] == 2018, (df['p_244']/df['p_12']),0)
df['p_244_c4'] = round((df['p_244_c1']+df['p_244_c2']+df['p_244_c3']),3)
df['p_244_ip'] = np.where(df['p_244_c4'] < 0, -99,df['p_244_c4'])
#Cálculo del gasto per-cápita por hogar a precio del 2018
df['p_232_c1'] = np.where(df['año'] == 2016, ((df['p_232']*(1 + 0.0575)*(1 + 0.0409))/df['p_12']),0)
df['p_232_c2'] = np.where(df['año'] == 2017, ((df['p_232']*(1 + 0.0409))/df['p_12']),0)
df['p_232_c3'] = np.where(df['año'] == 2018, (df['p_232']/df['p_12']),0)
df['p_232_c4'] = round((df['p_232_c1']+df['p_232_c2']+df['p_232_c3']),3)
df['p_232_gp'] = np.where(df['p_232_c4'] < 0, -99,df['p_232_c4'])

#Se actualiza a valor presente 2018 (recibido por concepto de trabajo)
df['p_112_c1'] = np.where(df['año'] == 2016, (df['p_112']*(1 + 0.0575)*(1 + 0.0409)),0)
df['p_112_c2'] = np.where(df['año'] == 2017, (df['p_112']*(1 + 0.0409)),0)
df['p_112_c3'] = np.where(df['año'] == 2018, df['p_112'],0)
df['p_112_c4'] = round((df['p_112_c1']+df['p_112_c2']+df['p_112_c3']),3)
df['p_112_vp'] = np.where(df['p_112_c4'] < 0, -99,df['p_112_c4'])
#Se actualiza a valor presente 2018 (recibido por concepto de arriendos)
df['p_113_c1'] = np.where(df['año'] == 2016, (df['p_113']*(1 + 0.0575)*(1 + 0.0409)),0)
df['p_113_c2'] = np.where(df['año'] == 2017, (df['p_113']*(1 + 0.0409)),0)
df['p_113_c3'] = np.where(df['año'] == 2018, df['p_113'],0)
df['p_113_c4'] = round((df['p_113_c1']+df['p_113_c2']+df['p_113_c3']),3)
df['p_113_vp'] = np.where(df['p_113_c4'] < 0, -99,df['p_113_c4'])
#Se actualiza a valor presente 2018 (recibido por concepto de pensiones o jubilaciones)
df['p_114_c1'] = np.where(df['año'] == 2016, (df['p_114']*(1 + 0.0575)*(1 + 0.0409)),0)
df['p_114_c2'] = np.where(df['año'] == 2017, (df['p_114']*(1 + 0.0409)),0)
df['p_114_c3'] = np.where(df['año'] == 2018, df['p_114'],0)
df['p_114_c4'] = round((df['p_114_c1']+df['p_114_c2']+df['p_114_c3']),3)
df['p_114_vp'] = np.where(df['p_114_c4'] < 0, -99,df['p_114_c4'])

#Se cambia por ceros:
df['p_227'] = np.where(df['p_227'] < 0, 0,df['p_227'])

#borrar temporales y dejar año y encuesta
df2 = df.drop(columns=['p_232_c1', 'p_232_c2', 'p_232_c3', 'p_232_c4', 'p_244_c1', 'p_244_c2', 'p_244_c3', 
                       'p_244_c4', 'p_112_c1', 'p_112_c2', 'p_112_c3', 'p_112_c4', 'p_113_c1', 'p_113_c2', 
                       'p_113_c3', 'p_113_c4', 'p_114_c1', 'p_114_c2', 'p_114_c3', 'p_114_c4',
                       'p_244','p_232','p_112','p_113','p_114',
                       'p_112_vp','p_113_vp','p_114_vp'
                       ]) # se actualizaron los valores de estas

df_final = df2
df_final

,año,encuesta,p_5,p_6,p_7,p_10,p_12,p_15a,p_15b,p_19a,p_19b,p_19c,p_19d,p_19e,p_20a,p_20b,p_36a,p_36b,p_43a,p_43b,p_43c,p_43d,p_44,p_45b,p_45c,p_45d,p_45e,p_45f,p_49b,p_49c,p_66b,p_66c,p_66d,p_67b,p_67c,p_68b,p_68c,p_83b,p_83c,p_83d,...,p_126,p_146,p_147,p_148,p_157,p_158,p_162,p_165,p_171,p_174,p_178,p_184,p_211,p_212,p_213,p_214,p_227,p_258,p_259,p_260,p_281,p_282,p_283,p_284,p_285,p_286,p_312,p_313,p_314,p_315,p_316,p_317,p_318,p_319,p_320,p_321,p_322,p_323,p_244_ip,p_232_gp
0,2016,12016.0,1,11,1108,5,6,0,6,0,0,2,3,1,6,0,2,4,0,2,0,0,1,1,0,4,1,0,5,1,3,3,0,3,3,3,3,0,0,1,...,-88,5,10,4,9,1,1,1,1,1,1,6,24,1,0,0,0,3,4,3,4,3,2,14,-88,5,3,3,3,4,4,4,4,4,3,4,3,4,0.000,-99.000
1,2017,12017.0,1,1,0101,2,4,3,1,0,0,2,2,0,2,2,2,2,0,2,0,0,4,0,0,3,0,0,4,0,0,0,4,0,4,0,4,0,0,0,...,0,4,7,4,3,1,1,1,1,1,1,6,9,0,1,0,0,1,3,3,4,1,2,1,7,19,1,1,2,2,1,2,3,3,2,3,3,3,153532.750,104090.000
2,2018,12018.0,1,3,0303,2,5,2,3,1,1,2,1,0,5,0,1,4,0,0,0,1,1,0,3,0,0,0,4,0,0,0,5,0,3,0,3,0,1,0,...,0,4,10,4,4,1,1,1,1,1,2,6,7,0,0,0,0,2,4,3,4,5,2,7,14,19,4,3,4,4,4,5,4,4,4,4,4,4,148000.000,240000.000
3,2016,22016.0,1,11,1104,4,2,1,1,0,0,1,1,0,2,0,1,1,0,0,0,1,1,0,1,1,0,0,0,2,1,1,0,1,1,1,1,0,0,0,...,0,4,10,4,4,1,1,1,1,1,1,6,13,0,0,1,900000,1,4,3,2,3,2,7,1,19,2,1,1,1,1,5,3,3,1,5,4,4,961066.353,1375939.688
4,2017,22017.0,1,1,0101,2,6,3,3,0,2,3,1,0,6,0,4,2,0,1,1,2,2,3,0,2,0,0,6,0,0,6,0,0,6,0,6,0,0,0,...,-88,5,10,4,3,1,1,2,1,1,2,4,8,0,1,0,0,-98,-98,1,4,3,2,7,14,1,3,2,3,4,4,2,2,4,3,4,4,3,65056.250,69393.333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27607,2018,168852018.0,1,13,1301,4,4,2,2,1,0,0,2,1,2,2,1,3,0,0,1,0,1,0,1,2,0,0,3,1,3,1,0,2,1,2,1,0,0,1,...,0,4,10,4,6,1,1,1,1,1,1,6,13,1,1,2,590000,2,4,2,3,3,2,7,11,19,4,3,1,3,4,2,2,1,3,4,5,4,307500.000,375000.000
27608,2018,168982018.0,1,13,1301,4,5,2,3,0,1,1,1,2,3,2,1,4,0,1,0,0,3,1,0,3,1,0,3,2,4,1,0,2,2,2,2,0,0,1,...,0,4,10,4,5,1,1,1,1,1,1,6,17,0,1,0,253000,1,4,2,3,3,3,2,6,19,3,3,3,3,3,3,3,1,3,3,3,5,577000.000,500000.000
27609,2018,169452018.0,1,14,1408,5,1,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,...,0,4,10,4,5,1,1,1,1,1,1,6,11,1,0,0,1400000,3,4,2,3,4,1,2,11,19,4,4,1,5,4,4,4,4,3,3,4,5,1580000.000,100000.000
27610,2018,178842018.0,1,16,1609,3,3,2,1,1,0,1,0,1,3,0,1,2,0,0,1,0,1,0,0,1,1,0,2,1,2,1,0,1,1,1,1,0,1,0,...,0,4,10,4,6,1,1,1,1,1,1,6,11,0,0,1,900000,1,3,-98,2,2,2,14,-88,19,3,4,4,4,3,3,4,4,4,3,3,4,396666.667,700000.000


In [39]:
df_final.to_csv('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Data/Processed/T1_ECV_prep.csv', sep=";")